# Manhattan plot for the susie_small example

Since the credible set (CS) example comes from MiGA_SVZ_eQTL, we will generate a genome-wide Manhattan plot for this cohort only.
To account for the small sample size, marginal p-values were obtained using linear regression instead of `susieR::univariate_regression()`, which assume normal distribution.

In [1]:
# Load data
your_path <- "~/project/susie_small/"
small_data <- readRDS(paste0(your_path, "MiGA_eQTL.chr2_ENSG00000151694.univariate_data.rds"))

In [2]:
str(small_data[[1]])

List of 10
 $ residual_Y       :List of 4
  ..$ MiGA_GFM_eQTL: num [1:66, 1] -0.281 -0.2128 -0.3517 -0.6193 -0.0751 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:66] "14-005" "14-015" "14-029" "14-046" ...
  .. .. ..$ : NULL
  ..$ MiGA_GTS_eQTL: num [1:56, 1] -0.2417 -0.2457 0.0436 -0.3436 0.5409 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:56] "14-005" "14-015" "14-029" "14-051" ...
  .. .. ..$ : NULL
  ..$ MiGA_SVZ_eQTL: num [1:47, 1] -0.3912 -0.3301 -0.0192 0.3922 -0.2067 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:47] "16-003" "16-024" "16-033" "16-062" ...
  .. .. ..$ : NULL
  ..$ MiGA_THA_eQTL: num [1:56, 1] -0.4856 -0.2359 0.3201 -0.1123 0.0549 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:56] "15-021" "15-024" "15-032" "15-041" ...
  .. .. ..$ : NULL
 $ residual_X       :List of 4
  ..$ : num [1:66, 1:7900] -0.1002 -0.1439 -0.1025 -0.0203 0.0393 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : ch

In [153]:
## check the variant counts 
7900+7562+7430+7697

[1] 30589

In [3]:
## using linear regression, which conducts analysis with t distribution, to calculate the marginal p value
get_marginal_pvalues <- function(region_data) {
  results <- list()
  
  for (r in seq_along(region_data$residual_Y)) {
    
    # Extract phenotype residuals (vector) and genotype residuals (matrix)
    Y_resid <- as.numeric(region_data$residual_Y[[r]])
    X_resid <- region_data$residual_X[[r]]
    
    variant_ids <- colnames(X_resid)
    n <- nrow(X_resid)
    
    res_df <- data.frame(
      variant_id = variant_ids,
      beta = NA_real_,
      se = NA_real_,
      tval = NA_real_,
      pval = NA_real_,
      context = names(region_data$residual_Y)[r]
    )
    
    # Loop through variants (columns of X_resid)
    for (j in seq_along(variant_ids)) {
      geno <- X_resid[, j]
      
      # Simple linear regression: Y ~ geno
      fit <- lm(Y_resid ~ geno)
      coef_summary <- summary(fit)$coefficients
      
      # Extract beta, se, t, p for geno
      res_df$beta[j] <- coef_summary["geno", "Estimate"]
      res_df$se[j]   <- coef_summary["geno", "Std. Error"]
      res_df$tval[j] <- coef_summary["geno", "t value"]
      res_df$pval[j] <- coef_summary["geno", "Pr(>|t|)"]
    }
    
    results[[r]] <- res_df
  }
  
  dplyr::bind_rows(results)
}

# Example usage
region_results <- get_marginal_pvalues(small_data[[1]])

# Top hits
head(region_results[order(region_results$pval), ])


,variant_id,beta,se,tval,pval,context
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
27663,chr2:9565770:A:G,0.4660013,0.07511243,6.204051,8.014104e-08,MiGA_THA_eQTL
20345,chr2:9703940:G:C,-0.6056692,0.10010201,-6.050519,2.627086e-07,MiGA_SVZ_eQTL
20346,chr2:9704049:G:A,-0.6056692,0.10010201,-6.050519,2.627086e-07,MiGA_SVZ_eQTL
27667,chr2:9569566:T:C,0.4758946,0.08125381,5.856890,2.889687e-07,MiGA_THA_eQTL
27658,chr2:9563097:C:T,0.4610775,0.08152103,5.655932,6.036584e-07,MiGA_THA_eQTL
15217,chr2:10467571:C:G,-0.6087742,0.11026360,-5.521081,9.867809e-07,MiGA_GTS_eQTL


In [5]:
dim(region_results[order(region_results$pval), ])

[1] 30553     6

In [9]:
head(MiGA_SVZ_eQTL)

,variant_id,beta,se,tval,pval,context
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,chr2:6843217:G:A,0.01218673,0.1367432,0.08912131,0.9293810,MiGA_SVZ_eQTL
2,chr2:6843960:A:G,0.01218673,0.1367432,0.08912131,0.9293810,MiGA_SVZ_eQTL
3,chr2:6845038:G:C,0.21304179,0.1668000,1.27722860,0.2080724,MiGA_SVZ_eQTL
4,chr2:6848538:C:T,0.03780192,0.1538081,0.24577330,0.8069753,MiGA_SVZ_eQTL
5,chr2:6849889:C:T,0.30097580,0.2081486,1.44596565,0.1551161,MiGA_SVZ_eQTL
6,chr2:6854646:C:T,0.13177179,0.1499316,0.87887928,0.3841357,MiGA_SVZ_eQTL


In [8]:
MiGA_SVZ_eQTL = region_results |> dplyr::filter(context == "MiGA_SVZ_eQTL")

In [11]:
install.packages("CMplot", lib = "~/temp_R_libs")
library(CMplot, lib.loc = "~/temp_R_libs")

Much appreciate for using CMplot.

Full description, Bug report, Suggestion and the latest codes:

https://github.com/YinLiLin/CMplot



In [12]:
library(dplyr)
library(tidyr)
library(CMplot)


# --- Preprocess for CMplot ---
plot_df <- MiGA_SVZ_eQTL %>%
  separate(variant_id, into = c("Chr", "Position", "Ref", "Alt"), sep = ":", remove = FALSE) %>%
  mutate(
    Chr = gsub("chr", "", Chr),     # remove "chr" prefix
    Position = as.numeric(Position)
  ) %>%
  rename(SNP = variant_id, P = pval) %>%
  select(SNP, Chr, Position, P)

head(plot_df)


,SNP,Chr,Position,P
,<chr>,<chr>,<dbl>,<dbl>
1,chr2:6843217:G:A,2,6843217,0.9293810
2,chr2:6843960:A:G,2,6843960,0.9293810
3,chr2:6845038:G:C,2,6845038,0.2080724
4,chr2:6848538:C:T,2,6848538,0.8069753
5,chr2:6849889:C:T,2,6849889,0.1551161
6,chr2:6854646:C:T,2,6854646,0.3841357


In [15]:
rectangular manhattan plot
CMplot(
  plot_df,
  plot.type = "m",        # Manhattan
  LOG10 = TRUE,           # plot -log10(P)
  col = c("grey30","grey60"),
  threshold = c(5e-8, 1e-5),        # two thresholds
  threshold.col = c("red","blue"),  # colors for each threshold
  threshold.lty = c(1,2),           # line type for each threshold
  signal.col = "blue",
  file = "pdf",
  file.name = "manhattan_eqtl",
  width = 10, height = 6, dpi = 300,
  file.output = TRUE
)


 Rectangular Manhattan plotting P.
 Plots are stored in: /home/ubuntu/project/susie_small 


In [13]:
## Circular manhattan plot, fancier
CMplot(
  plot_df,
  plot.type = "c",   # circular
  LOG10 = TRUE,
  r = 0.4, 
  outward = FALSE,
  col = c("dodgerblue3","goldenrod2"),
  threshold = 5e-8,
  threshold.col = "red",
  threshold.lty = 2,
  file = "pdf",
  file.name = "manhattan_eqtl_circular",
  width = 10, height = 10, dpi = 300,
  file.output = TRUE
)

 Circular Manhattan plotting P.


Warning message in CMplot(plot_df, plot.type = "c", LOG10 = TRUE, r = 0.4, outward = FALSE, :
“No significant points for P pass the threshold level using threshold=5e-08!”


 Plots are stored in: /home/ubuntu/project/susie_small 
